# [ Pré-processamento ] Heurísticas computacionais para extração deconhecimento em problemas de maratona deprogramação - Parte 2 (PLN)

Após realizar o tratamento básico dos dados, é necessário realizar alguns passos básicos de Processamento de Linguagem Natural (PLN) a fim de poder fornecer os problemas como entrada para algum classificador.

### Importando as bibliotecas necessárias

Antes de começar, é necessário importar algumas bibliotecas para tratar os dados.

In [1]:
import os
import json
import nltk
import scipy
import graphviz
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from scipy.spatial import distance

### Declarando constantes de trabalho

Criação de "variáveis de ambiente" para auxiliar a obtenção de dados.

In [2]:
# Langs? {PT, EN, ES}
LANG_SLUG = "EN"
GRAPHS_OUTPUT_DIR = "../Graphs/"
PREPROCESSED_DATA_DIR = "../Datasets/"

# Preprocessed files {%LANG_00_raw_data, %LANG_01_primary_data, %LANG_02_secondary_data, %LANG_03_terciary_data}.csv
PREPROCESSED_FILE = "%s%s%s" % (PREPROCESSED_DATA_DIR, LANG_SLUG, "_03_terciary_data.csv")

### Tokenização

Aqui os textos de descrição, entrada e saída serão tokenizados (extraídos as palavras)

In [3]:
# Lê o arquivo CSV e trata os objetos vazios
rawDataDF = pd.read_csv(PREPROCESSED_FILE, sep=";", index_col = 0)
rawDataDF.fillna("", inplace = True)

# Tokeniza os textos do problema
tokenized = pd.DataFrame()
tokenized["problem"] = rawDataDF.apply(lambda x : nltk.word_tokenize(x["description"]), axis = 1)
tokenized["in"] = rawDataDF.apply(lambda x : nltk.word_tokenize(x["input"]), axis = 1)
tokenized["out"] = rawDataDF.apply(lambda x : nltk.word_tokenize(x["output"]), axis = 1)

# Converte todas as letras para "minúsculas"
tokenized["problem"] = tokenized.apply(lambda x: [w.lower() for w in x["problem"]], axis = 1)
tokenized["in"] = tokenized.apply(lambda x: [w.lower() for w in x["in"]], axis = 1)
tokenized["out"] = tokenized.apply(lambda x: [w.lower() for w in x["out"]], axis = 1)

# Imprime parte do DF
tokenized.head()

,problem,in,out
1001,"[read, 2, variables, ,, named, a, and, b, and,...","[the, input, file, will, contain, 2, integer, ...","[print, the, letter, x, (, uppercase, ), with,..."
1002,"[the, formula, to, calculate, the, area, of, a...","[the, input, contains, a, value, of, floating,...","[present, the, message, ``, a=, '', followed, ..."
1003,"[read, two, integer, values, ,, in, this, case...","[the, input, file, contains, 2, integer, numbe...","[print, the, variable, soma, with, all, the, c..."
1004,"[read, two, integer, values, ., after, this, ,...","[the, input, file, contains, 2, integer, numbe...","[print, prod, according, to, the, following, e..."
1005,"[read, two, floating, points, ', values, of, d...","[the, input, file, contains, 2, floating, poin...","[print, media, (, average, in, portuguese, ), ..."


### Remoção de stopwords

Remoção de stopwords e limpeza de caracteres (remoção de palavras não alfanuméricas) com o auxílio do NLTK

In [4]:
stoplist = stopwords.words('english')

tokenized["problem"] = tokenized.apply(lambda x: [w for w in x["problem"] if (w not in stoplist and w.isalpha())], axis = 1)
tokenized["in"] = tokenized.apply(lambda x: [w for w in x["in"] if (w not in stoplist and w.isalpha())], axis = 1)
tokenized["out"] = tokenized.apply(lambda x: [w for w in x["out"] if (w not in stoplist and w.isalpha())], axis = 1)

# Imprime parte do DF
tokenized.head()

,problem,in,out
1001,"[read, variables, named, b, make, sum, two, va...","[input, file, contain, integer, numbers]","[print, letter, x, uppercase, blank, space, eq..."
1002,"[formula, calculate, area, circumference, defi...","[input, contains, value, floating, point, doub...","[present, message, followed, value, variable, ..."
1003,"[read, two, integer, values, case, variables, ...","[input, file, contains, integer, numbers]","[print, variable, soma, capital, letters, blan..."
1004,"[read, two, integer, values, calculate, produc...","[input, file, contains, integer, numbers]","[print, prod, according, following, example, b..."
1005,"[read, two, floating, points, values, double, ...","[input, file, contains, floating, points, valu...","[print, media, average, portuguese, according,..."


### Aplicação de tag - Part Of Speech (POS)

Aplicação de POS tags utilizando a NLTK

In [5]:
# Aplica o POS_TAG
tagged = pd.DataFrame()
tagged["problem"] = tokenized.apply(lambda x: nltk.pos_tag(x["problem"]), axis = 1)
tagged["in"] = tokenized.apply(lambda x: nltk.pos_tag(x["in"]), axis = 1)
tagged["out"] = tokenized.apply(lambda x: nltk.pos_tag(x["out"]), axis = 1)

# Imprime parte do DF
tagged.head()

,problem,in,out
1001,"[(read, JJ), (variables, NNS), (named, VBN), (...","[(input, NN), (file, NN), (contain, NN), (inte...","[(print, NN), (letter, NN), (x, NNP), (upperca..."
1002,"[(formula, NN), (calculate, NN), (area, NN), (...","[(input, NN), (contains, VBZ), (value, NN), (f...","[(present, JJ), (message, NN), (followed, VBD)..."
1003,"[(read, VB), (two, CD), (integer, JJR), (value...","[(input, NN), (file, NN), (contains, VBZ), (in...","[(print, NN), (variable, JJ), (soma, NN), (cap..."
1004,"[(read, VB), (two, CD), (integer, JJR), (value...","[(input, NN), (file, NN), (contains, VBZ), (in...","[(print, NN), (prod, NN), (according, VBG), (f..."
1005,"[(read, VB), (two, CD), (floating, VBG), (poin...","[(input, NN), (file, NN), (contains, VBZ), (fl...","[(print, NN), (media, NNS), (average, JJ), (po..."


### Aplicação do Lemmatizador

Conversão das palavras para suas formas inflexionadas utilizando o WordNetLemmatizer. **Ex.:** "following" -> "follow".

Caso o idioma escolhido seja o português, deve-se utilizar algum lemmatizador alternativo ou então o próprio stemmer para a língua portuguesa disponível na NLTK.

In [6]:
# Convert POS to WordNet POS
# From https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#wordnetlemmatizer
def to_wn_pos(tagged_word):
    tag_dict = {"J" : nltk.corpus.wordnet.ADJ,
                "N" : nltk.corpus.wordnet.NOUN,
                "V" : nltk.corpus.wordnet.VERB,
                "R" : nltk.corpus.wordnet.ADV}
    return tag_dict.get(tagged_word[1][0].upper(), nltk.wordnet.NOUN)

# Aplica a lemmatização
lemmatizer = nltk.stem.WordNetLemmatizer()

lemmatized = pd.DataFrame()
lemmatized["problem"] = tagged.apply(lambda x: [(lemmatizer.lemmatize(tup[0], to_wn_pos(tup)), tup[1]) for tup in x["problem"]], axis = 1)
lemmatized["in"] = tagged.apply(lambda x: [(lemmatizer.lemmatize(tup[0], to_wn_pos(tup)), tup[1]) for tup in x["in"]], axis = 1)
lemmatized["out"] = tagged.apply(lambda x: [(lemmatizer.lemmatize(tup[0], to_wn_pos(tup)), tup[1]) for tup in x["out"]], axis = 1)

# Imprime parte do DF
lemmatized.head()

,problem,in,out
1001,"[(read, JJ), (variable, NNS), (name, VBN), (b,...","[(input, NN), (file, NN), (contain, NN), (inte...","[(print, NN), (letter, NN), (x, NNP), (upperca..."
1002,"[(formula, NN), (calculate, NN), (area, NN), (...","[(input, NN), (contain, VBZ), (value, NN), (fl...","[(present, JJ), (message, NN), (follow, VBD), ..."
1003,"[(read, VB), (two, CD), (integer, JJR), (value...","[(input, NN), (file, NN), (contain, VBZ), (int...","[(print, NN), (variable, JJ), (soma, NN), (cap..."
1004,"[(read, VB), (two, CD), (integer, JJR), (value...","[(input, NN), (file, NN), (contain, VBZ), (int...","[(print, NN), (prod, NN), (accord, VBG), (foll..."
1005,"[(read, VB), (two, CD), (float, VBG), (point, ...","[(input, NN), (file, NN), (contain, VBZ), (flo...","[(print, NN), (medium, NNS), (average, JJ), (p..."


### Geração da matriz termo-documento

Como o objetivo deste trabalho é realizar a comparação entre a semântica dos problemas, então, torna-se mais viável tratar os problemas como sendo as linhas da matriz e as palavras (termos) como sendo as colunas (para possíveis fins de compactação posterior). **ATENÇÃO:** Por motivos de desempenho, as linhas da matriz termo-documento serão compostas pelo vocabulário e as colunas pelos problemas. Caso seja necessário, basta obter o DF transposto.

#### Análise do vocabulário

In [7]:
# Vocabulário dos problemas, entrada e saída
vocab = {
    "problem" : set([]),
    "in" : set([]),
    "out" : set([])
}

lemmatized.apply(lambda x : vocab["problem"].update(set([[] if a == [] else a[0] for a in x["problem"]])), axis = 1)
lemmatized.apply(lambda x : vocab["in"].update(set([[] if a == [] else a[0] for a in x["in"]])), axis = 1)
lemmatized.apply(lambda x : vocab["out"].update(set([[] if a == [] else a[0] for a in x["out"]])), axis = 1)

# Dimensões do dicionário de vocabulários
print("Vocabulary dict shape :")
print(len(vocab["problem"]), len(vocab["in"]), len(vocab["out"]))
print("\nProblems (first ones):")
print(list(vocab["problem"])[:10])
print("\nInput (first ones):")
print(list(vocab["in"])[:10])
print("\nOutput (first ones):")
print(list(vocab["out"])[:10])

Vocabulary dict shape :
14252 4559 3931

Problems (first ones):
['tamanho', 'sunflower', 'renè', 'importância', 'rematched', 'flaw', 'rabbit', 'ação', 'consistency', 'surprised']

Input (first ones):
['tamanho', 'thus', 'vine', 'prix', 'topmost', 'rabbit', 'atk', 'ação', 'balloon', 'binding']

Output (first ones):
['tamanho', 'thus', 'sunflower', 'vine', 'mom', 'atk', 'balloon', 'minimize', 'th', 'carry']


#### Construção, efetiva, da matriz

In [8]:
# Procedimento definido para preencher a "matriz" termo-documento
def parse_matrix_rows(row):
    column = str(row.name)
    problem_document_term_mat[column] = 0
    in_document_term_mat[column] = 0
    out_document_term_mat[column] = 0
    
    for tup in row["problem"]:
        problem_document_term_mat[column].loc[tup[0]] += 1
    for tup in row["in"]:
        in_document_term_mat[column].loc[tup[0]] += 1
    for tup in row["out"]:
        out_document_term_mat[column].loc[tup[0]] += 1
    


problem_document_term_mat = pd.DataFrame(index = vocab["problem"])
in_document_term_mat = pd.DataFrame(index = vocab["in"])
out_document_term_mat = pd.DataFrame(index = vocab["out"])

# Construção da "matriz" termo-documento (ou documento-termo)
lemmatized.apply(parse_matrix_rows, axis = 1)

# Imprime as dimensões das matrizes
print("\nProblems: ", problem_document_term_mat.shape)
print("\nInput: ", in_document_term_mat.shape)
print("\nOutput: ", out_document_term_mat.shape)
print("\n\n")

# Imprime parte do df de problemas
problem_document_term_mat.head()


Problems:  (14252, 1833)

Input:  (4559, 1833)

Output:  (3931, 1833)





,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,2952,2953,2954,2955,2956,2957,2958,2959,2960,2961
tamanho,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sunflower,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
renè,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
importância,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rematched,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Geração do grafo de similaridade semântica

Para a análise de similaridade entre textos, dever-se-á calcular a distância entre os vetores-coluna da matriz. Neste caso, será utilizada a distância do cosseno que, convenientemente, já é normalizada (a função cosseno só retorna valores entre -1 e 1, sendo que, como todos os vetores só posssuem valores positivos, ela retornará valores entre 0 e 1).

In [10]:
# Calcula a distância do cosseno entre os problemas
# Quanto mais próximo de 1 (pela função da biblioteca), mais distantes estão entre si

problems_i = problem_document_term_mat.columns
graph_path = "%s%s" % (GRAPHS_OUTPUT_DIR, "Graph_1.gv")
graph = graphviz.Graph(name="Graph 1", filename = graph_path, format = "png")
added_vertex = []

for index_n, problem_i in enumerate(problems_i):
    for s_problem_i in problems_i[index_n + 1 : len(problems_i) - 1]:
        dist = distance.cosine(problem_document_term_mat[problem_i].values, problem_document_term_mat[s_problem_i].values)
        if dist < 0.3 :
            if problem_i not in added_vertex:
                graph.node(problem_i)
                added_vertex.append(problem_i)
            if s_problem_i not in added_vertex:
                graph.node(s_problem_i)
                added_vertex.append(s_problem_i)
            graph.edge(problem_i, s_problem_i, label = str(dist))

graph.render()

'../Graphs\\Graph_1.gv.png'